# بهینه‌سازی مدل MLP برای تشخیص بیماری قلبی
در این نوت‌بوک، مدل MLP را با استفاده از RandomizedSearchCV بهینه می‌کنیم.

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD, Adam
from scikeras.wrappers import KerasClassifier
from scipy.stats import randint, uniform


In [20]:
# بارگذاری و پیش‌پردازش داده‌ها
df = pd.read_csv('Heart_disease_cleveland_new.csv')
X = df.drop('target', axis=1)
y = df['target']

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

X_train = X_train
y_train = y_train.values


In [21]:
def build_model(hidden_layers=1, neurons=[32], learning_rate=0.01, activation='relu', optimizer='SGD'):
    model = Sequential()
    
    # اضافه کردن لایه‌های پنهان
    for i in range(hidden_layers):
        if i == 0:
            # لایه ورودی
            model.add(Dense(neurons[i], input_dim=X_train.shape[1], activation=activation))
        else:
            # لایه‌های میانی
            model.add(Dense(neurons[i], activation=activation))
    
    # لایه خروجی
    model.add(Dense(1, activation='sigmoid'))
    
    # انتخاب بهینه‌ساز
    if optimizer == 'SGD':
        opt = SGD(learning_rate=learning_rate)
    elif optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    else:
        raise ValueError("Optimizer must be 'SGD' or 'adam'")
    
    # کامپایل مدل
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# تعریف KerasClassifier
model = KerasClassifier(
    model=build_model,
    hidden_layers=1,
    neurons=[32],
    learning_rate=0.01,
    activation='relu',
    optimizer='SGD',
    verbose=0
)

In [22]:
#تعریف پارامترها
param_distributions = {
    'hidden_layers': [1, 2],
    'neurons': [[32], [64, 32], [16], [32, 16]],
    'learning_rate': uniform(0.001, 0.01),
    'activation': ['relu', 'tanh'],
    'epochs': [20, 30],
    'batch_size': [16, 32, 64],
    'optimizer': ['SGD', 'adam']
}



In [23]:
# تنظیم RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=model,
                                   param_distributions=param_distributions,
                                   n_iter=20,
                                   scoring='accuracy',
                                   cv=3,
                                   verbose=1,
                                   n_jobs=1,
                                   random_state=42)


random_search_result = random_search.fit(X_train, y_train)

Fitting 3 folds for each of 20 candidates, totalling 60 fits


c:\Users\Emad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Emad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\Emad\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in th

In [24]:
# بهترین پارامترها
print(f"Best Accuracy: {random_search_result.best_score_}")
print(f"Best Parameters: {random_search_result.best_params_}")


Best Accuracy: 0.8098251028806583
Best Parameters: {'activation': 'tanh', 'batch_size': 16, 'epochs': 30, 'hidden_layers': 1, 'learning_rate': 0.009500385777897993, 'neurons': [32, 16], 'optimizer': 'adam'}


In [25]:
# نمایش نتایج
results_df = pd.DataFrame(random_search_result.cv_results_)
results_df = results_df.sort_values(by='mean_test_score', ascending=False)
results_df[['params', 'mean_test_score', 'std_test_score']].head(10)

,params,mean_test_score,std_test_score
18,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.809825,0.021859
14,"{'activation': 'relu', 'batch_size': 16, 'epoc...",0.797428,0.016641
2,"{'activation': 'tanh', 'batch_size': 64, 'epoc...",0.793364,0.015679
12,"{'activation': 'tanh', 'batch_size': 32, 'epoc...",0.776852,0.001309
7,"{'activation': 'relu', 'batch_size': 32, 'epoc...",0.768621,0.004510
0,"{'activation': 'relu', 'batch_size': 16, 'epoc...",0.768570,0.015718
6,"{'activation': 'relu', 'batch_size': 16, 'epoc...",0.764558,0.019099
10,"{'activation': 'tanh', 'batch_size': 32, 'epoc...",0.756173,0.015738
1,"{'activation': 'tanh', 'batch_size': 64, 'epoc...",0.739763,0.034249
4,"{'activation': 'tanh', 'batch_size': 16, 'epoc...",0.739403,0.057607
